# Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Navigate to Classification folder

In [ ]:
%%bash
cd /content/drive/MyDrive/Classification
pwd

/content/drive/MyDrive/Classification


# Install dependencies

In [ ]:
pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.4/891.4 kB 45.8 MB/s eta 0:00:00


# Train Dataset

In [ ]:
"""
ThinhDV  11 Nov 2024
How to train/fine-tune a pre-trained model on a custom dataset (i.e., transfer learning)
"""
import torch
from torch import nn, save, load
from tqdm import tqdm
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchmetrics.functional import accuracy
from torchvision.transforms import ToTensor, Resize
import numpy as np

from torch import nn

# Image Classifier Neural Network
class Alexnet(nn.Module):
    def __init__(self,num_classes=100):
        super(Alexnet, self).__init__()
        self.model = nn.Sequential(
            #### Convolutional Layers ####
            #input: 224*224*3
            # Layer 1:
            nn.Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4)),  # Change 1 to 3 for RGB images: output = 32
            #output: W = (224-11+2*0)/4 + 1=54, H = (224-11+2*0)/4 + 1=54
            nn.ReLU(),  # output:96, W=H=54 do co cung chieu dai moi canh sau conv
            nn.BatchNorm2d(96),
            nn.MaxPool2d(2, 2),
            # after pooling: ((W=H=54-kernel=2)/stride=2) + 1= 27; nn.Flatten(), nn.Linear(96*(27)*(27), 53),

            # Layer 2
            nn.Conv2d(96, 256, kernel_size=(5, 5), padding=(2, 2)),  # input: 96, output: 256
            #output: W=(27 - 5 + 2 * 2) / 1 + 1 = 26, H=(27 - 5 + 2 * 2) / 1 + 1 = 26
            nn.ReLU(),  # output:256, W=H=26 do co cung chieu dai moi canh sau conv
            nn.BatchNorm2d(256),
            nn.MaxPool2d(3, 2),
            # after pooling: ((W=H=26-kernel=3)/stride=2)+1 = 12; nn.Flatten(), nn.Linear(256*(12)*(12), 53),

            # Layer 3
            nn.Conv2d(256, 384, kernel_size=(3, 3), padding=(1, 1)),  # input: 256, output: 384
            # output: W=(12 - 3 + 2 * 1) / 1 + 1 = 12, W=(12 - 3 + 2 * 1) / 1 + 1 = 12
            nn.ReLU(),  # output:384, W=H=12 do co cung chieu dai moi canh sau conv
            nn.BatchNorm2d(384),

            # Layer 4
            nn.Conv2d(384, 384, kernel_size=(3, 3), padding=(1, 1)),  # input: 384, output: 384
            # output: W=(12 - 3 + 2 * 1) / 1 + 1 = 12, W=(12 - 3 + 2 * 1) / 1 + 1 = 12
            nn.ReLU(),  # output:384, W=H=12 do co cung chieu dai moi canh sau conv
            nn.BatchNorm2d(384),

            # Layer 5
            nn.Conv2d(384, 256, kernel_size=(3, 3), padding=(1, 1)),  # input: 384, output: 256
            # output: W=(12 - 3 + 2 * 1) / 1 + 1 = 12, W=(12 - 3 + 2 * 1) / 1 + 1 = 12
            nn.ReLU(),  # output:256, W=H=12 do co cung chieu dai moi canh sau conv
            nn.BatchNorm2d(256),
            nn.MaxPool2d(3, 2),
            # after pooling: ((W=H=12-kernel=3)/stride=2)+1 = 6; nn.Flatten(), nn.Linear(256*(6)*(6), 53),

            #### Fully-Connected Layer ####
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(256*6 *6, num_classes), #tinh sao ra bang 6??????
        )
        #Truyen tham so vao
        self.num_classes = num_classes

    def forward(self, x):
        return self.model(x)


# Setup CUDA
def setup_cuda():
    # Setting seeds for reproducibility
    seed = 50
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

    return torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


def train_model():
    """
    Train the model over a single epoch
    :return: training loss and training accuracy
    """
    train_loss = 0.0
    train_acc = 0.0
    model.train()

    for (img, label) in tqdm(train_loader, ncols=80, desc='Training'):
        # Get a batch
        img, label = img.to(device, dtype=torch.float), label.to(device, dtype=torch.long)

        # Set the gradients to zero before starting backpropagation
        optimizer.zero_grad()

        # Perform a feed-forward pass
        logits = model(img)

        # Compute the batch loss
        loss = loss_fn(logits, label)

        # Compute gradient of the loss fn w.r.t the trainable weights
        loss.backward()

        # Update the trainable weights
        optimizer.step()

        # Accumulate the batch loss
        train_loss += loss.item()

        # Get the predictions to calculate the accuracy for every iteration. Remember to accumulate the accuracy
        prediction = logits.argmax(axis=1)
        train_acc += accuracy(prediction, label, task='multiclass', average='macro', num_classes=len(class_names)).item()

    return train_loss / len(train_loader), train_acc / len(train_loader)


def validate_model():
    """
    Validate the model over a single epoch
    :return: validation loss and validation accuracy
    """
    model.eval()
    valid_loss = 0.0
    val_acc = 0.0

    with torch.no_grad():
        for (img, label) in tqdm(val_loader, ncols=80, desc='Valid'):
            # Get a batch
            img, label = img.to(device, dtype=torch.float), label.to(device, dtype=torch.long)

            # Perform a feed-forward pass
            logits = model(img)

            # Compute the batch loss
            loss = loss_fn(logits, label)

            # Accumulate the batch loss
            valid_loss += loss.item()

            # Get the predictions to calculate the accuracy for every iteration. Remember to accumulate the accuracy
            prediction = logits.argmax(axis=1)
            val_acc += accuracy(prediction, label, task='multiclass', average='macro', num_classes=len(class_names)).item()

    return valid_loss / len(val_loader), val_acc / len(val_loader)


if __name__ == "__main__":
    device = setup_cuda()

    # 1. Load the dataset
    transform = transforms.Compose([Resize((224, 224)), ToTensor()])
    train_dataset = ImageFolder(root='/content/drive/MyDrive/dataset/playcards/train', transform=transform)
    val_dataset = ImageFolder(root='/content/drive/MyDrive/dataset/playcards/valid', transform=transform)
    # Get class names
    class_names = train_dataset.classes

    # 2. Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

    # 3. Create a new deep model with pre-trained weights
    model = Alexnet(
        num_classes=len(class_names),
    ).to(device)

    # 4. Specify loss function and optimizer
    optimizer = Adam(model.parameters(), lr=1e-4)
    loss_fn = torch.nn.CrossEntropyLoss()

    # 5. Train the model with 100 epochs
    max_acc = 0
    for epoch in range(100):

        # 5.1. Train the model over a single epoch
        train_loss, train_acc = train_model()

        # 5.2. Validate the model after training
        val_loss, val_acc = validate_model()

        print(f'Epoch {epoch}: Validation loss = {val_loss}, Validation accuracy: {val_acc}')

        # 4.3. Save the model if the validation accuracy is increasing
        if val_acc > max_acc:
            print(f'Validation accuracy increased ({max_acc} --> {val_acc}). Model saved')
            max_acc = val_acc
            torch.save(model.state_dict(), 'epoch_' + str(epoch) + '_acc_{0:.4f}'.format(max_acc) + '.pt')



Valid: 100%|██████████████████████████████████████| 1/1 [00:14<00:00, 14.22s/it]


Epoch 0: Validation loss = 1.364609956741333, Validation accuracy: 0.3500000238418579
Validation accuracy increased (0 --> 0.3500000238418579). Model saved


Valid: 100%|██████████████████████████████████████| 1/1 [00:00<00:00, 11.85it/s]


Epoch 1: Validation loss = 1.1601483821868896, Validation accuracy: 0.45000001788139343
Validation accuracy increased (0.3500000238418579 --> 0.45000001788139343). Model saved


Valid: 100%|██████████████████████████████████████| 1/1 [00:00<00:00, 10.81it/s]


...

Epoch 97: Validation loss = 1.4054460525512695, Validation accuracy: 0.6500000357627869


Valid: 100%|██████████████████████████████████████| 1/1 [00:00<00:00,  8.96it/s]


Epoch 98: Validation loss = 1.4210631847381592, Validation accuracy: 0.6500000357627869


Valid: 100%|██████████████████████████████████████| 1/1 [00:00<00:00, 11.05it/s]

Epoch 99: Validation loss = 1.287003755569458, Validation accuracy: 0.6500000357627869


# Testing

In [ ]:
import torch
from torch import nn, save, load
from tqdm import tqdm
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchmetrics.functional import accuracy
from torchvision.transforms import ToTensor, Resize
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt

# Thiết lập biến cần thiết
train_dir = '/content/drive/MyDrive/dataset/playcards/train'
test_dir = '/content/drive/MyDrive/dataset/playcards/test'
valid_dir = '/content/drive/MyDrive/dataset/playcards/valid'
NUM_WORKERS = os.cpu_count()
BATCH_SIZE = 32
IMG_SIZE = 224
manual_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])
patch_size = 16
CLASS = {
    0: 'ace_of_clubs',
    1: 'ace_of_diamonds',
    2: 'ace_of_hearts',
    3: 'ace_of_spades',
}


# Thiết lập thiết bị (GPU hoặc CPU)
def setup_cuda():
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    return device


device = setup_cuda()

# Thiết lập các transform để xử lý ảnh
manual_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])


# Hàm dự đoán kết quả cho một hình ảnh
def predict_image(image_path, model, transform, class_names, device):
    model.eval()
    img = Image.open(image_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted_class = torch.max(output, 1)
    predicted_label = class_names[predicted_class.item()]
    return img, predicted_label


# Hàm chính để dự đoán các hình ảnh trong tập test
def test_model():
    # 1. Tải dữ liệu và lớp từ tập train
    transform = transforms.Compose([Resize((224, 224)), ToTensor()])
    train_dataset = ImageFolder(root=train_dir, transform=transform)
    test_dataset = ImageFolder(root=test_dir, transform=transform)
    # Get class names
    class_names = train_dataset.classes

    # 2. Tạo mô hình ViT và tải trạng thái từ checkpoint
    model = Alexnet(
        num_classes=len(class_names),
    ).to(device)

    folder_checkpoint = '/content/'  # Define the folder name
    file_checkpoint = os.path.join(folder_checkpoint, 'epoch_8_acc_0.7500.pt')  # lay epoch cuoi cung
    model.load_state_dict(torch.load(file_checkpoint, device))
    print('Model loaded from checkpoint.')
    # Ensure the output directory exists
    output_dir = "result"
    os.makedirs(output_dir, exist_ok=True)

    # 3. Dự đoán kết quả cho mỗi hình ảnh trong tập test
    for image_path in tqdm(test_dataset.imgs, desc='Testing'):
        img, predicted_label = predict_image(image_path[0], model, manual_transforms, class_names, device)
        # plt.imshow(img)
        # plt.title(f'Predicted: {predicted_label}')
        # plt.show()

        # Convert the tensor image back to a PIL image if necessary
        if isinstance(img, torch.Tensor):
            img = transforms.ToPILImage()(img)

        # Create a plot
        fig, ax = plt.subplots()

        # Set white background
        fig.patch.set_facecolor('white')
        ax.set_facecolor('white')

        # Remove axis
        ax.axis('off')

        # Display the image
        ax.imshow(img)

        # Add the predicted label as the title
        ax.set_title(f'Predicted: {predicted_label}', fontsize=12, pad=10)

        # Save the figure
        image_basename = os.path.basename(image_path[0])
        image_name, image_ext = os.path.splitext(image_basename)
        output_image_path = os.path.join(output_dir, f"{image_name}_pred_{predicted_label}.png")

        plt.savefig(output_image_path, bbox_inches='tight', pad_inches=0.1)
        plt.close(fig)


if __name__ == '__main__':
    test_model()

<ipython-input-6-6b2a8a065cfa>:80: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(file_checkpoint, device))


Model loaded from checkpoint.


Testing: 100%|██████████| 20/20 [00:20<00:00,  1.02s/it]
